### Import Packages

In [1]:
from datetime import datetime as dt
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Outline the Model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 200

# Clear from memory the variables from previous runs. Variables = weights and biases
tf.reset_default_graph()

# placeholders for inputs and targets - what type and shape are they
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

# Shape of the weights matrix is: w = k * m, where k = number of input nodes, and m = number of hidden units
# Shape of the biases matrix is: b = 1 * m
weights_1 = tf.get_variable("weights_1",[input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

# tf.nn is a module that contains neural network functions including activitation functions
# this line applies an activitation function to the linear combination of the inputs, weights and biases
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# Define the second hidden layer
weights_2 = tf.get_variable("weights_2", [hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2 = tf.nn.tanh(tf.matmul(outputs_1,weights_2) + biases_2)

#Define the output layer
weights_3 = tf.get_variable("weights_3", [hidden_layer_size,output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

outputs = tf.matmul(outputs_2, weights_3) + biases_3

#logits are the unscaled probabilities. Softmax will create scaled probabilities - a vector summing to 1.
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)

#Mean loss gives a big performance boost
#reduce mean takes the mean of elements across a tensor
mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(mean_loss)

out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

#SGD batch size is 1. GD is # of samples. We want somewhere in between to get balance between speed and accuracy
batch_size = 100

batches_number = mnist.train._num_examples // batch_size

max_epochs = 40

prev_validation_loss = 9999999.

train_start_time = dt.now()

for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0.
    
    for batch_counter in range(batches_number):
        
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict={inputs: input_batch, targets: target_batch})

        curr_epoch_loss += batch_loss

    #outer loop! get mean loss
    curr_epoch_loss /= batches_number

    #Get the validation data
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    #Calculate the validation loss and accuracy for the epoch by doing forward propagation
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
                                                   feed_dict={inputs: input_batch, targets: target_batch})
    
    print('Epoch '+str(epoch_counter+1)+
        '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
        '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
        '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%'       
         )
    
    #Early stopping - if validation loss starts to increase - to prevent overfitting
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss

train_end_time = dt.now()
train_delta = train_end_time - train_start_time

print('End of training! Training time(s): '+'{0:.2f}'.format(train_delta.total_seconds()))


Epoch 1. Training loss: 0.686. Validation loss: 0.303. Validation accuracy: 91.94%
Epoch 2. Training loss: 0.270. Validation loss: 0.217. Validation accuracy: 93.84%
Epoch 3. Training loss: 0.208. Validation loss: 0.177. Validation accuracy: 94.94%
Epoch 4. Training loss: 0.171. Validation loss: 0.149. Validation accuracy: 95.90%
Epoch 5. Training loss: 0.145. Validation loss: 0.133. Validation accuracy: 96.20%
Epoch 6. Training loss: 0.125. Validation loss: 0.118. Validation accuracy: 96.70%
Epoch 7. Training loss: 0.109. Validation loss: 0.109. Validation accuracy: 96.82%
Epoch 8. Training loss: 0.096. Validation loss: 0.100. Validation accuracy: 97.00%
Epoch 9. Training loss: 0.085. Validation loss: 0.093. Validation accuracy: 97.22%
Epoch 10. Training loss: 0.076. Validation loss: 0.088. Validation accuracy: 97.44%
Epoch 11. Training loss: 0.067. Validation loss: 0.083. Validation accuracy: 97.62%
Epoch 12. Training loss: 0.060. Validation loss: 0.077. Validation accuracy: 97.64%
E

### Test

In [5]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)

test_accuracy = sess.run([accuracy],
                        feed_dict={inputs: input_batch, targets: target_batch })

test_accuracy_percent = test_accuracy[0] * 100

print('Test accuracy is: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy is: 97.76%
